In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
import cartopy.crs as ccrs
import ipywidgets as widgets

In [2]:
# Konstanten
GM      = 3.986e14                  # [km^3/s^2]
OMEGA_E = 7.292115e-5               # rad*s^(-1)
X_P     = -0.1*np.pi/(180*60*60)    # rad
Y_P     = 0.4*np.pi/(180*60*60)     # rad

def R1(a):
    return np.array(
        [
            [1, 0, 0],
            [0, np.cos(a), np.sin(a)],
            [0, -np.sin(a), np.cos(a)],
        ]
    )

def R2(a):
    return np.array(
        [
            [np.cos(a), 0, -np.sin(a)],
            [0, 1, 0],
            [np.sin(a), 0, np.cos(a)],
        ]
    )

def R3(a):
    return np.array(
        [
            [np.cos(a), np.sin(a), 0],
            [-np.sin(a), np.cos(a), 0],
            [0, 0, 1],
        ]
    )

def to_rad(*aa):
    return [a*np.pi/180 for a in aa]


def position(dt, a, i, e, omega, M0, Omega):
    i, omega, M0, Omega = to_rad(i, omega, M0, Omega)

    # Berechnungen
    n = (GM / a**3)**(1/2)
    p = a*(1-e**2)

    M = n*dt + M0

    cnt = 0
    E, prev_E = M, None
    
    while E != prev_E and cnt < 100_000:
        E, prev_E = M + e*np.sin(E), E
        cnt += 1
    
    v = 2*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2))
    r = p/(1 + e*np.cos(v))

    rb_vec = np.array([r*np.cos(v), r*np.sin(v), 0])

    r_vec = R3(-Omega)@R1(-i)@R3(-omega)@rb_vec

    return r_vec

def true_earth_fixed(coords, t):
    n = len(coords)
    return np.array([
        R3(time*OMEGA_E*t/(n-1))@coord
        for time, coord in enumerate(coords)
    ])

def conv_earth_fixed(coords):
    return np.array([
        R2(-X_P)@R1(-Y_P)@coord
        for coord in coords
    ])

def to_ground_track(coords):
    return np.array([
        [
            np.arctan2(coord[1], coord[0]),
            np.arctan2(coord[2], np.sqrt(coord[0]**2 + coord[1]**2))
        ]
        for coord in coords
    ])

def lat_formatter(x, pos):
    direction = ""
    if 0 < x < 180:
        direction = "E"
    elif -180 < x < 0:
        direction = "W"
    
    return f"{abs(x)}°" + direction

def long_formatter(x, pos):
    direction = ""
    if x > 0:
        direction = "N"
    elif x < 0:
        direction = "S"
    
    return f"{abs(x)}°" + direction

def remove_lines(coords):
    coords_copy = np.copy(coords)
    cnt = 0
    for i, c in enumerate(coords_copy):
        if abs(c[0]-coords_copy[i-1,0]) > np.pi:
            coords = np.insert(coords, i+cnt, np.array([np.nan, np.nan]), axis=0)
            cnt += 1
    return coords

def plot(sat):
    plt.close('Figure 1')
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 4), subplot_kw=dict(projection=ccrs.PlateCarree()))
    ax.set_xticks(np.arange(-180, 181, 30))
    ax.set_yticks(np.arange(-90, 91, 30))
    ax.xaxis.set_major_formatter(FuncFormatter(lat_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(long_formatter))
    ax.coastlines(color='lightgray')
    plt.plot(
        sat[:,0], 
        sat[:,1],
        color="Blue"
    )

    plt.axis('scaled')
    plt.xlim([-180, 180])
    plt.ylim([-90, 90])
    plt.title('Bodenspur der Satelliten')
    fig.set_label('Figure 1')
    plt.xlabel('Länge')
    plt.ylabel('Breite')
    plt.tight_layout()
    plt.show()

sat = None
params = None

def inter(a, i, e, omega, M0, Omega):
    sat = np.array([position(dt, a, i, e, omega, M0, Omega) for dt in range(0, 86401, 600)])
    sat = true_earth_fixed(sat, 86400)
    sat = conv_earth_fixed(sat)
    sat = to_ground_track(sat)
    sat = remove_lines(sat)*180/np.pi

    plot(sat)

widgets.interactive(
    inter,
    a = widgets.IntSlider(min=8_000_00, max=50_000_000, step=1_000, value=42_164_000), 
    i = widgets.IntSlider(min=-360, max=360, step=1, value=55), 
    e = widgets.FloatSlider(min=0, max=1, step=0.05, value=0.3), 
    omega = widgets.IntSlider(min=-360, max=360, step=1, value=270), 
    M0 = widgets.IntSlider(min=-360, max=360, step=1, value=180), 
    Omega = widgets.IntSlider(min=-360, max=360, step=1, value=-70)
)

interactive(children=(IntSlider(value=42164000, description='a', max=50000000, min=800000, step=1000), IntSlid…